In [1]:
import pandas as pd

pricesDataFolder = "../prices_data/"
dfDaily = pd.read_csv(
    filepath_or_buffer=pricesDataFolder+"D1/[SP500]_Daily_2014-03-20__2024-03-20.csv",
    delimiter="\t"
)
dfDaily = dfDaily.rename(
    columns = {
        '<DATE>':'date', 
        '<TIME>':'time', 
        '<OPEN>':'open', 
        '<HIGH>':'high', 
        '<LOW>':'low', 
        '<CLOSE>':'close'
    }
)
dfDaily = dfDaily.drop(['<TICKVOL>', '<VOL>', '<SPREAD>'], axis=1)

In [2]:
import pandas as pd
import sys; sys.path.insert(1, '..')
import utility
pricesDataFolder = "../prices_data/"
dfH4 = pd.read_csv(
    filepath_or_buffer=pricesDataFolder+"H4/[SP500]_H4_2014-03-20__2024-03-20.csv",
    delimiter="\t"
)

dfH4 = dfH4.rename(
    columns = {
        '<OPEN>':'open', 
        '<HIGH>':'high', 
        '<LOW>':'low', 
        '<CLOSE>':'close'
    }
)

dfH4['datetime'] = pd.to_datetime(dfH4['<DATE>'] + ' ' + dfH4['<TIME>'])
dfH4 = dfH4.drop(['<TICKVOL>', '<VOL>', '<SPREAD>', '<DATE>', '<TIME>'], axis=1)
dfH4 = utility.heikinashi(dfH4) #add heikin ashi candles
dfH4 = utility.addHACandleColor(dfH4)

In [6]:
# from dataclasses import dataclass
# @dataclass
# class test:
#     a:bool
#     b:float
#     c:int

# test1 = test(True, 1.2, 4)
# test2 = test(False, 56.8, 2)
# test3 = test(True, 1.2, 4)

# print(test1 == test2)
# print(test1 == test3)

False
True


In [4]:
from dataclasses import dataclass

@dataclass
class HyperParamaters:
    useFilter:bool
    filterPercent:float
    clusterBW:int
    useSR:bool
    useUpdateSl:bool
    pivotN1N2:tuple[int,int]
    shortTermMA:int
    longTermMA:int
    uselongTermMA:bool
 

In [5]:
from strategies.Strategy import Strategy
from key_levels_finder.key_levels_algorithm import getKeyLevels
import strategy_tester

class Scorer:
    def __init__(self, strategy:Strategy, dfDaily: pd.DataFrame, dfH4: pd.DataFrame):
        self.strategy = strategy
        self.dfDaily = dfDaily
        self.dfH4 = dfH4
        self.keyLevelsParametersAlreadyTested: dict[tuple[bool, float, int], list[float]] = {}
        self.hyperparametersByScore: dict[float, HyperParamaters] = {}

    def giveStrategyScore(self, useFilter, filterPercent, clusterBW, useSR, useUpdateSl, pivotN1N2, shortTermMA, longTermMA, uselongTermMA):
            
        if uselongTermMA: self.dfH4["longTermMA"] = self.dfH4["close"].rolling(window=longTermMA).mean() # add moving average long
        self.dfH4["shortTermMA"] = self.dfH4["close"].rolling(window=shortTermMA).mean() # add moving average short

        keyLevels: list[float]
        keyLevelsParameters = (useFilter, filterPercent, clusterBW)
        if keyLevelsParameters in self.keyLevelsParametersAlreadyTested:
            keyLevels = self.keyLevelsParametersAlreadyTested[keyLevelsParameters]
        else:
            keyLevels = getKeyLevels(self.dfDaily, pivotN1N2=pivotN1N2, useFilter=False, filterPercent=3, clusterBW=30)
            self.keyLevelsParametersAlreadyTested[keyLevelsParameters] = keyLevels

        self.strategy.setParams(keyLevels=keyLevels, useSR=useSR, useUpdateSl=useUpdateSl, longTermMAPeriod=longTermMA, uselongTermMA=uselongTermMA)
        trades = strategy_tester.strategyLoop(self.dfH4, self.strategy, longTermMA)    
        profitEachMonth = trades.groupby(pd.Grouper(key='exit_date', axis=0, freq='ME'))["profit"].sum()
        hyperParameters = HyperParamaters(useFilter, filterPercent, clusterBW, useSR, useUpdateSl, pivotN1N2, shortTermMA, longTermMA, uselongTermMA)
        
        capitalAfterTradeEachMonth = trades.groupby(pd.Grouper(key='exit_date', axis=0, freq='ME'))["capital_after_trade"].sum()
        capitalBeforeTradeEachMonths = capitalAfterTradeEachMonth - profitEachMonth
        
        percentEachMonth = ((capitalAfterTradeEachMonth - capitalBeforeTradeEachMonths)/capitalBeforeTradeEachMonths)*100

        self.hyperparametersByScore[percentEachMonth.mean()] = hyperParameters

In [6]:
from strategies.Heikin_Ashi_Moving_Average_Strategy import HeikinAshiMovingAverage

# hyperparameters
useFilterParams = useSRParams = useUpdateSlParams = [False, True]
filterPercentParams = [4.0, 5.0, 6.0, 7.0]
clusterBWParams = [80, 90, 100, 110, 120, 130]
pivotN1N2Params = [(10,10), (10,20), (20,10), (20,20)]
shortTermMAParams = [30, 50, 60, 80, 100]

MyScorer = Scorer(strategy=HeikinAshiMovingAverage(), dfDaily=dfDaily, dfH4=dfH4)    

for useFilter in useFilterParams:
    for filterPercent in filterPercentParams:
        for clusterBW in clusterBWParams:
            for useSR in useSRParams:
                for useUpdateSl in useUpdateSlParams:
                    for pivotN1N2 in pivotN1N2Params:
                        for shortTermMA in shortTermMAParams:
                            MyScorer.giveStrategyScore(useFilter, filterPercent, clusterBW, useSR, useUpdateSl, pivotN1N2, shortTermMA, longTermMA=200, uselongTermMA=False)

print(MyScorer.hyperparametersByScore)


{0.5769754326883941: HyperParamaters(useFilter=True, filterPercent=7.0, clusterBW=130, useSR=False, useUpdateSl=False, pivotN1N2=(20, 20), shortTermMA=30, longTermMA=200, uselongTermMA=False), 0.4890195346879243: HyperParamaters(useFilter=True, filterPercent=7.0, clusterBW=130, useSR=False, useUpdateSl=False, pivotN1N2=(20, 20), shortTermMA=50, longTermMA=200, uselongTermMA=False), 0.4902056189009038: HyperParamaters(useFilter=True, filterPercent=7.0, clusterBW=130, useSR=False, useUpdateSl=False, pivotN1N2=(20, 20), shortTermMA=60, longTermMA=200, uselongTermMA=False), 0.40783554874610706: HyperParamaters(useFilter=True, filterPercent=7.0, clusterBW=130, useSR=False, useUpdateSl=False, pivotN1N2=(20, 20), shortTermMA=80, longTermMA=200, uselongTermMA=False), 0.38733430776094424: HyperParamaters(useFilter=True, filterPercent=7.0, clusterBW=130, useSR=False, useUpdateSl=False, pivotN1N2=(20, 20), shortTermMA=100, longTermMA=200, uselongTermMA=False), 0.4335325050326754: HyperParamaters(

In [8]:

myKeys = list(MyScorer.hyperparametersByScore.keys())
myKeys.sort()
myKeys

[0.21540297704118203,
 0.2924275625589604,
 0.33632765787105756,
 0.3503938463178055,
 0.3622429677556697,
 0.362723017495203,
 0.36323463425766056,
 0.37801179748863206,
 0.38733430776094424,
 0.39266092900935523,
 0.40783554874610706,
 0.4335325050326754,
 0.4374200817302261,
 0.4537047805594508,
 0.4890195346879243,
 0.4902056189009038,
 0.4988352407890395,
 0.5075222018856193,
 0.5769754326883941,
 0.5800249243621987]

In [9]:
for key in myKeys: 
    print(MyScorer.hyperparametersByScore[key])

HyperParamaters(useFilter=True, filterPercent=7.0, clusterBW=130, useSR=False, useUpdateSl=True, pivotN1N2=(20, 20), shortTermMA=80, longTermMA=200, uselongTermMA=False)
HyperParamaters(useFilter=True, filterPercent=7.0, clusterBW=130, useSR=False, useUpdateSl=True, pivotN1N2=(20, 20), shortTermMA=100, longTermMA=200, uselongTermMA=False)
HyperParamaters(useFilter=True, filterPercent=7.0, clusterBW=130, useSR=False, useUpdateSl=True, pivotN1N2=(20, 20), shortTermMA=50, longTermMA=200, uselongTermMA=False)
HyperParamaters(useFilter=True, filterPercent=7.0, clusterBW=130, useSR=True, useUpdateSl=False, pivotN1N2=(20, 20), shortTermMA=80, longTermMA=200, uselongTermMA=False)
HyperParamaters(useFilter=True, filterPercent=7.0, clusterBW=130, useSR=True, useUpdateSl=True, pivotN1N2=(20, 20), shortTermMA=100, longTermMA=200, uselongTermMA=False)
HyperParamaters(useFilter=True, filterPercent=7.0, clusterBW=130, useSR=False, useUpdateSl=True, pivotN1N2=(20, 20), shortTermMA=60, longTermMA=200, 

In [13]:
print(MyScorer.hyperparametersByScore[0.5773884232345082])
print(MyScorer.hyperparametersByScore[0.5761017449887972])
print(MyScorer.hyperparametersByScore[0.5346332652047951])
print(MyScorer.hyperparametersByScore[0.5312571603744237])


HyperParamaters(useFilter=True, filterPercent=5.0, clusterBW=90, useSR=True, useUpdateSl=True, longTermMA=150, uselongTermMA=False)
HyperParamaters(useFilter=True, filterPercent=5.0, clusterBW=90, useSR=True, useUpdateSl=True, longTermMA=200, uselongTermMA=False)
HyperParamaters(useFilter=True, filterPercent=5.0, clusterBW=90, useSR=True, useUpdateSl=False, longTermMA=200, uselongTermMA=False)
HyperParamaters(useFilter=True, filterPercent=5.0, clusterBW=90, useSR=True, useUpdateSl=True, longTermMA=300, uselongTermMA=False)
